In [5]:
import pandas as pd

In [6]:
import os
os.getcwd()

'C:\\Users\\Work\\Desktop\\github\\CodeLou\\PythonDataAnalysis'

In [7]:
df = pd.read_excel(r".\FSIS-Recall-Summary-2014.xlsx")


In [8]:
df.sample(10)

,CY 2014 Recalls,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
86,2014-12-22 00:00:00,086-2014,III,Hickory Smoked Sliced Bacon,Undeclared Substance,80050
18,2014-03-07 00:00:00,018-2014,I,Chicken Products,Undeclared Allergen,29200
75,2014-10-30 00:00:00,075-2014,I,Boudin Products,Other,25765
72,2014-10-24 00:00:00,072-2014,II,Salad with Chicken,Undeclared Allergen,377
21,2014-04-04 00:00:00,021-2014,II,Chicken Nugget Products,Extraneous Material,75320
71,2014-10-17 00:00:00,071-2014,I,Lamb Products,Other,27948
31,2014-05-21 00:00:00,031-2014,I,Chicken Breast and Tender Products,Undeclared Allergen,23250
6,2014-01-19 00:00:00,006-2014,II,Cheeseburger Mac Products,Undeclared Allergen,1770000
33,2014-05-29 00:00:00,033-2014,III,Boneless Skinless Chicken Breasts,Other,363
77,2014-11-09 00:00:00,077-2014,II,Frozen Pork Products,Undeclared Allergen,4465
